# Illustration of extracellular potentials and current conservation


In [ ]:
%matplotlib inline
import numpy as np
import LFPy
import neuron
h = neuron.h
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from brainsignals.plotting_convention import mark_subplots, simplify_axes, cmap_v_e
from brainsignals import neural_simulations as ns

np.random.seed(1234)


# Define synapse parameters
synapse_params = {
    'e' : 0.,                   # reversal potential
    'syntype' : 'Exp2Syn',       # synapse type
    'tau1' : 0.1,                 # synaptic time constant
    'tau2' : 1.,                 # synaptic time constant
    #'weight' : 0.001,            # synaptic weight
    'record_current' : True,    # record synapse current
}

grid_x, grid_z = np.mgrid[-550:551:15, -500:1501:25]
grid_y = np.zeros(grid_x.shape)

# Define electrode parameters
grid_elec_params = {
    'sigma': 0.3,      # extracellular conductivity
    'x': grid_x.flatten(),  # electrode requires 1d vector of positions
    'y': grid_y.flatten(),
    'z': grid_z.flatten()
}

def return_two_comp_cell(tstop, dt):

    h("forall delete_section()")
    h("""
    proc celldef() {
      topol()
      subsets()
      geom()
      biophys()
      geom_nseg()
    }

    create soma[1], dend[1]

    proc topol() { local i
      basic_shape()
      connect dend(0), soma(1)
    }
    proc basic_shape() {
      soma[0] {pt3dclear()
      pt3dadd(0, 0, 0.0, 2)
      pt3dadd(0, 0, 500., 2)}
      dend[0] {pt3dclear()
      pt3dadd(0, 0, 500.0, 2)
      pt3dadd(0, 0, 1000.0, 2)}
    }

    objref all
    proc subsets() { local i
      objref all
      all = new SectionList()
        soma[0] all.append()
        dend[0] all.append()

    }
    proc geom() {
    }
    proc geom_nseg() {
    soma[0] {nseg = 1}
    dend[0] {nseg = 1}
    }
    proc biophys() {
    }
    celldef()

    Ra = 150.
    cm = 1.
    Rm = 30000.

    forall {
        insert pas // 'pas' for passive, 'hh' for Hodgkin-Huxley
        g_pas = 1 / Rm
        Ra = Ra
        cm = cm
        }
    """)
    cell_params = {
                'morphology': h.all,
                'delete_sections': False,
                'v_init': -70.,
                'passive': False,
                'nsegs_method': None,
                'dt': dt,
                'tstart': -100.,
                'tstop': tstop,
                'pt3d': True,
            }
    cell = LFPy.Cell(**cell_params)
    cell.set_pos(z=-cell.z[0].mean() + 200)
    return cell


In [ ]:
cell_names = ["two_comp", "hay"]

cells = []
LFPs = []

for cell_name in cell_names:
    
    if cell_name == "hay":
        cell = ns.return_hay_cell(tstop=5, dt=2**-4, make_passive=True)
        synapse_params["idx"] = cell.get_closest_idx(x=100, z=1400)
        synapse_params["weight"] = 0.001
        # Create a grid of measurement locations, in (mum)

        
    else:
        cell = return_two_comp_cell(tstop=5, dt=2**-4)
        synapse_params["idx"] = cell.get_closest_idx(x=100, z=1400)
        synapse_params["weight"] = 0.001


    synapse = LFPy.Synapse(cell, **synapse_params)
    synapse.set_spike_times(np.array([1.]))

    # Run simulation, electrode object argument in cell.simulate
    print("running simulation...")
    cell.simulate(rec_imem=True, rec_vmem=True)
    print("max dV: ", np.max(np.abs(cell.vmem[:, :, None] - cell.vmem[:, 0])))
    grid_electrode = LFPy.RecExtElectrode(cell, **grid_elec_params)
    LFP = 1e6 * grid_electrode.get_transformation_matrix() @ cell.imem

    LFP -= LFP[:, 0, None]
    time_idx = np.argmax(np.abs(cell.imem[0, :]))

    LFP = LFP[:, time_idx].reshape(grid_x.shape)
    LFPs.append(LFP)
    cells.append(cell)
    

In [ ]:

plt.close("all")
fig = plt.figure(figsize=[6, 4/5 * 6])
fig.subplots_adjust(top=0.97, bottom=0.05, right=0.98, 
                    left=-0.00, wspace=-0.05, hspace=0.)

xlim = [np.min(grid_x), np.max(grid_x)]
ylim = [np.min(grid_z), np.max(grid_z)]



num = 15
levels = np.logspace(-2.2, 0, num=num)
scale_max = 300# 10**np.ceil(np.log10(np.max(np.abs(grid_LFP)))) / 1
levels_norm = scale_max * np.concatenate((-levels[::-1], levels))


colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2))
                   for i in range(len(levels_norm) -1)]
colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)

titles = {"hay": "multi-compartment\nneuron",
          "two_comp": "two-compartment\nneuron"}

ax_1 = fig.add_subplot(2, 3, 1, xticks=[], yticks=[], xlim=xlim, ylim=ylim,
                  aspect=1, frameon=False,)
ax_1.set_title("point neuron", y=0.85)
ax_2 = fig.add_subplot(2, 3, 4, xticks=[], yticks=[], xlim=xlim, ylim=ylim,
                  aspect=1, frameon=False,)

theta = np.linspace(0, 2 * np.pi, 20)
ax_1.plot(100 * np.cos(theta), 100 * np.sin(theta) + 500, c='gray', zorder=0)
ax_2.plot(100 * np.cos(theta), 100 * np.sin(theta) + 500, c='gray', zorder=0)

ax_1.arrow(150, 520, -100, 0, color='b', head_width=20)
ax_1.arrow(50, 480, 100, 0, color='r', head_width=20)

ax_2.text(0, 1000, "no extracellular\npotential\n" + r"V$_e$=0", ha='center')

mark_subplots(ax_1, "A", ypos=1.0, xpos=0.0)

for n in range(len(cells)):
    cell = cells[n]
    LFP = LFPs[n]

    print(scale_max, np.max(np.abs(LFP)))
    
    ax_1 = fig.add_subplot(2, 3, 2 + n, xticks=[], yticks=[], xlim=xlim, ylim=ylim,
                      aspect=1, frameon=False,)
    ax_1.set_title(titles[cell_names[n]], y=0.85)
    ax_2 = fig.add_subplot(2, 3, 5 + n, xticks=[], yticks=[], xlim=xlim, ylim=ylim,
                      aspect=1, frameon=False,)

    if n == 0:
        ax_1.text(250, 370, "two currents of\nequal magnitude\nI$_1$=-I$_2$")
        ax_2.text(450, 400, "dipolar\nextracellular\npotential")

        ax_1.arrow(200, 750, -150, 0, color='b', head_width=20)
        ax_1.arrow(0, 250, 150, 0, color='r', head_width=20)
        lw = 3
    else:
        ax_1.text(300, 330, "big input\ncurrent\n\ndistributed\nreturn\ncurrents")
        ax_1.arrow(220 + 150, 1170, -150, 0, color='b', head_width=20)
        ax_1.arrow(60, 850, 50, 0, color='r', head_width=20)
        ax_1.arrow(20, 600, 50, 0, color='r', head_width=20)
        ax_1.arrow(20, 350, 50, 0, color='r', head_width=20)


        ax_2.text(250, 400, "complex\n(dipolar-like)\nextracellular\npotential")
        lw = 1
    ax_1.plot(cell.x.T, cell.z.T, lw=lw, c='gray', solid_capstyle="round", zorder=0)
    ax_2.plot(cell.x.T, cell.z.T, lw=lw, c='gray', solid_capstyle="round")

    ep_intervals = ax_2.contourf(grid_x, grid_z, LFP,
                               zorder=-2, colors=colors_from_map,
                               levels=levels_norm, extend='both')

    ax_2.contour(grid_x, grid_z, LFP, colors='k', linewidths=(1), zorder=-2,
               levels=levels_norm)

    mark_subplots(ax_1, "BC"[n], ypos=1.0, xpos=-0.15)


cax = fig.add_axes([0.6, 0.1, 0.3, 0.01], frameon=False)

cbar = fig.colorbar(ep_intervals, cax=cax, orientation='horizontal',
             #format='%.0E', 
                    extend='max')

cbar.set_label(r'V$_{\rm e}$ (nV)', labelpad=-0)
cbar.set_ticks([-100, -10, 0, 10, 100])


plt.savefig('illustration_EP.pdf')
